# 01_data_checks — فحوصات أولية

In [7]:

import pandas as pd, numpy as np, json, os
with open("/workspace/data/project_config.json") as f:
    CFG = json.load(f)
RAW = "/workspace/data/ML_Trading_DataLayer_v1.xlsx"
df = pd.read_excel(RAW)
print(df.shape)
df.head()

(6, 8)


,Provider,Markets,Data Types,Realtime,Timeframes,Delivery,Corporate Actions,Notes
0,Yahoo Finance,US/Global (broad),"EOD, Historical",No (primarily delayed),"1D, 1W, 1M (basic intraday via 1m historical w...",REST/Download/CSV,Splits/Dividends (adjusted),Good for prototyping/backtesting; licensing re...
1,Polygon.io,US Equities/Options/FX/Crypto,"Realtime, Historical, Aggregates, Tick",Yes (WebSocket/Low-latency),"Tick, 1s, 1m, 5m, 15m, 1h, 1D",REST/WebSocket,"Yes (splits, dividends, symbol changes)",Developer-friendly; good docs; tiered pricing
2,Refinitiv (LSEG),Global multi-asset,"Low-latency realtime, Historical, Fundamentals",Yes (ultra-low latency options),Tick to 1D+,APIs/Feeds/SDK,Comprehensive (global),Institutional-grade; higher cost; licensing pe...
3,Tadawul (Saudi Exchange),Saudi equities,"Realtime/Delayed, Historical",Yes (depending on license),1m to 1D (tick via vendors),Official feeds / partners,Official disclosures/corporate actions,Must review licensing; ideal for KSA deployment
4,Tiingo,US Equities/ETF/FX (some global),"EOD, Intraday, Fundamentals",Yes (with paid plans),1m to 1D (plus aggregates),REST,Yes (splits/dividends),Clean historical datasets; sensible pricing fo...


In [8]:

print("Columns:", df.columns.tolist())
display(df.dtypes)
display(df.describe(include='all').T.iloc[:20])


Columns: ['Provider', 'Markets', 'Data Types', 'Realtime', 'Timeframes', 'Delivery', 'Corporate Actions', 'Notes']


Provider             object
Markets              object
Data Types           object
Realtime             object
Timeframes           object
Delivery             object
Corporate Actions    object
Notes                object
dtype: object

,count,unique,top,freq
Provider,6,6,Yahoo Finance,1
Markets,6,6,US/Global (broad),1
Data Types,6,6,"EOD, Historical",1
Realtime,6,6,No (primarily delayed),1
Timeframes,6,5,Tick to 1D+,2
Delivery,6,6,REST/Download/CSV,1
Corporate Actions,6,6,Splits/Dividends (adjusted),1
Notes,6,6,Good for prototyping/backtesting; licensing re...,1


In [9]:

nulls = df.isnull().sum().sort_values(ascending=False)
display(nulls[nulls>0])
dups = df.duplicated().sum()
print("Duplicates:", dups)


Series([], dtype: int64)

Duplicates: 0


In [10]:

# تحويل تواريخ إن وجدت + حذف تكرارات كاملة
for c in df.columns:
    if "date" in c.lower() or "time" in c.lower():
        try: df[c] = pd.to_datetime(df[c], errors="coerce")
        except: pass
df_clean = df.drop_duplicates().copy()
print("After dedup:", df_clean.shape)


After dedup: (6, 8)


/tmp/ipykernel_1829/785978011.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  try: df[c] = pd.to_datetime(df[c], errors="coerce")
/tmp/ipykernel_1829/785978011.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  try: df[c] = pd.to_datetime(df[c], errors="coerce")


In [11]:

num_cols = df_clean.select_dtypes(include=[np.number]).columns
iqr_stats = {}
for c in num_cols:
    Q1, Q3 = df_clean[c].quantile(0.25), df_clean[c].quantile(0.75)
    IQR = Q3 - Q1
    low, high = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    iqr_stats[c] = {"low": float(low), "high": float(high),
                    "outliers": int(((df_clean[c]<low)|(df_clean[c]>high)).sum())}
iqr_stats


{}

In [12]:

with open("/workspace/data/project_config.json") as f:
    CFG = json.load(f)
out_path = f"{CFG['paths']['processed']}/ML_Trading_DataLayer_clean.parquet"
df_clean.to_parquet(out_path, index=False)
print("Saved:", out_path, "→", len(df_clean), "rows")


Saved: /workspace/data/processed/ML_Trading_DataLayer_clean.parquet → 6 rows


In [13]:
import os, pandas as pd
clean_path = "/workspace/data/processed/ML_Trading_DataLayer_clean.parquet"
print("Exists:", os.path.exists(clean_path), "→", clean_path)
if os.path.exists(clean_path):
    dfc = pd.read_parquet(clean_path)
    print("Shape:", dfc.shape)
    display(dfc.head())


Exists: True → /workspace/data/processed/ML_Trading_DataLayer_clean.parquet
Shape: (6, 8)


,Provider,Markets,Data Types,Realtime,Timeframes,Delivery,Corporate Actions,Notes
0,Yahoo Finance,US/Global (broad),"EOD, Historical",NaT,NaT,REST/Download/CSV,Splits/Dividends (adjusted),Good for prototyping/backtesting; licensing re...
1,Polygon.io,US Equities/Options/FX/Crypto,"Realtime, Historical, Aggregates, Tick",NaT,NaT,REST/WebSocket,"Yes (splits, dividends, symbol changes)",Developer-friendly; good docs; tiered pricing
2,Refinitiv (LSEG),Global multi-asset,"Low-latency realtime, Historical, Fundamentals",NaT,NaT,APIs/Feeds/SDK,Comprehensive (global),Institutional-grade; higher cost; licensing pe...
3,Tadawul (Saudi Exchange),Saudi equities,"Realtime/Delayed, Historical",NaT,NaT,Official feeds / partners,Official disclosures/corporate actions,Must review licensing; ideal for KSA deployment
4,Tiingo,US Equities/ETF/FX (some global),"EOD, Intraday, Fundamentals",NaT,NaT,REST,Yes (splits/dividends),Clean historical datasets; sensible pricing fo...
